# Llama-FinSent-S
## Fine-Tuning & Merge.
This notebook starts with the oopere/pruned40-llama-3.2-1B model and applies a fine-tuning process using LoRA to create a model capable of analyzing sentiment in financial texts.

________
* Model: oopere/pruned40-llama-3.2-1B
* Dataset: FinGPT/fingpt-sentiment-train
_________
This notebook simply replicates the process followed to create the Llama-FinSent-S model.

If you’re looking for explanations of each step, you can find them in the Pruning section of the [Large Language Models course](https://github.com/peremartra/Large-Language-Model-Notebooks-Course) that I maintain on GitHub.

# Install and Import Libraries.

In [2]:
!pip install -q transformers==4.47.1
!pip install -q datasets==3.2.0
!pip install -q torch==2.5.1
!pip install -q lm-eval==0.4.7
!pip install -q peft==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [3]:
# Import Libraries (updated)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from transformers import DataCollatorForLanguageModeling

# Load & Trat the Dataset

In [4]:
# Load the Dataset
dataset = load_dataset("FinGPT/fingpt-sentiment-train", split="train")

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

(…)-00000-of-00001-dabab110260ac909.parquet:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/76772 [00:00<?, ? examples/s]

In [24]:
dataset['input'][0]

'Teollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .'

In [28]:
dataset['output'][0]

'neutral'

In [30]:
dataset['instruction'][0]

'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.'

## Load Tokenizer & Model

In [5]:
# Load the Base Model and Tokenizer.
model_name = "oopere/pruned40-llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=4916, bias=False)
          (up_proj): Linear(in_features=2048, out_features=4916, bias=False)
          (down_proj): Linear(in_features=4916, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

## Format Prompt for Training.

In [7]:
newline_token_id = tokenizer.encode("\n", add_special_tokens=False)[0]  # Get token ID for newline

def format_prompt_for_training(example):
    instruction = example.get("instruction", "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.")
    input_text = example.get("input", example.get("text", ""))
    # Append the output (label) as the target (if available)
    target = f" {example['output']}" if "output" in example and example["output"] is not None else ""
    full_text = f"Instruction: {instruction}\nInput: {input_text}\nAnswer:{target}\n"
    return {"text": full_text}

In [8]:
# Apply formatting
formatted_dataset = dataset.map(format_prompt_for_training)
train_dataset = formatted_dataset

Map:   0%|          | 0/76772 [00:00<?, ? examples/s]

## Tokenize Dataset.

In [9]:
# Tokenization
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        # Remove padding here - let collator handle it dynamically
    )

tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

Map:   0%|          | 0/76772 [00:00<?, ? examples/s]

# Training

In [10]:
# LoRA Configuration (unchanged)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
model_peft = get_peft_model(model, lora_config)

In [11]:
print(model_peft.print_trainable_parameters())

trainable params: 4,378,112 || all params: 918,148,608 || trainable%: 0.4768
None


In [12]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./lora_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=16,
    learning_rate=3e-4,
    fp16=True,
    weight_decay=0.01,
    warmup_steps=1000,
    save_steps=500,
    logging_steps=10,
    evaluation_strategy="no",
    report_to="none",
    # Add this to handle padding properly:
    dataloader_pin_memory=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Use proper data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model_peft,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    data_collator=data_collator,
)

In [14]:
# Execute Training
trainer.train()
print("Training complete!")

Step,Training Loss
10,4.993900
20,4.896200
30,4.608700
40,4.221900
50,3.647100
60,3.033300
70,2.639300
80,2.446500
90,2.347900
100,2.292500


Training complete!


## Save & Upload Model to HF

In [18]:
new_model_name = "llama-3.2-1B-financial-sentiment"

# Merge the LoRA Adapter with the base model.
merged_model = model_peft.merge_and_unload()
merged_model.save_pretrained(new_model_name)

In [17]:
from huggingface_hub import HfApi

In [19]:
hf_username = "eagle0504"  # Replace with your Hugging Face username
repo_name = new_model_name  # Change this as needed
repo_id = f"{hf_username}/{repo_name}"

api = HfApi()

# Create the repository
api.create_repo(repo_id=repo_id, exist_ok=True)  # exist_ok=True prevents errors if the repo already exists

print(f"Repository created: https://huggingface.co/{repo_id}")

Repository created: https://huggingface.co/eagle0504/llama-3.2-1B-financial-sentiment


In [20]:
merged_model.push_to_hub(
    new_model_name,
    private=True,
    use_temp_dir=False
)

tokenizer.push_to_hub(
    new_model_name,
    private=True,
    use_temp_dir=False
)

model.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/eagle0504/llama-3.2-1B-financial-sentiment/commit/5c4a61581051ee960e8a2afcaff05189aecb5eac', commit_message='Upload tokenizer', commit_description='', oid='5c4a61581051ee960e8a2afcaff05189aecb5eac', pr_url=None, repo_url=RepoUrl('https://huggingface.co/eagle0504/llama-3.2-1B-financial-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='eagle0504/llama-3.2-1B-financial-sentiment'), pr_revision=None, pr_num=None)

# Conclusions

Our financial sentiment analysis model is now ready.

The entire process took just over 40 minutes on a Google Colab A100 GPU, which is roughly equivalent to an RTX 3090.
